In [6]:

import pandas as pd

df_model = pd.read_csv(r"C:\\Users\\JellevanAs\\Documents\\Studie\\Thesis\\oude sets\\df-empty.csv")

In [1]:
# Now, we'll download a pre-trained Word2Vec model using gensim
import gensim.downloader as api

# Load the Word2Vec model (this might take some time)
word2vec_model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [7]:
print(df_model.columns.tolist())
df_model = df_model.drop(df_model.columns[0], axis=1)

['Unnamed: 0', 'url', 'price', 'address', 'descrip', 'listed_since', 'zip_code', 'size', 'year', 'living_area', 'kind_of_house', 'building_type', 'num_of_rooms', 'num_of_bathrooms', 'layout', 'energy_label', 'insulation', 'heating', 'ownership', 'exteriors', 'parking', 'date_list', 'last_ask_price', 'last_ask_price_m2', 'city', 'log_id', 'num of tokens per descrip', 'descrip_en', 'numerical_price', 'numerical_price_per_m2', 'tag', 'house_category', 'living_area_float', 'size_float', 'zip_code_4_digits', 'postcode', 'latitude', 'longitude']


In [14]:
import math
def numberToWords(n):
    if not isinstance(n, int):
        raise ValueError("Input must be an integer")

    # Handle negative numbers
    if n < 0:
        return "minus " + numberToWords(-n)

    limit, t = 1000000000000, 0

    # If zero, return 'zero'
    if (n == 0):
        return "zero"

    # Arrays for various number parts
    multiplier = ["", "Trillion", "Billion", "Million", "Thousand"]
    first_twenty = ["", "One", "Two", "Three", "Four", "Five", "Six", "Seven", "Eight", "Nine", "Ten", "Eleven", "Twelve", "Thirteen", "Fourteen", "Fifteen", "Sixteen", "Seventeen", "Eighteen", "Nineteen"]
    tens = ["", "Twenty", "Thirty", "Forty", "Fifty", "Sixty", "Seventy", "Eighty", "Ninety"]

    # If number is less than 20, return the corresponding word
    if (n < 20):
        return first_twenty[n]

    answer = ""
    i = n
    while(i > 0):
        curr_hun = i // limit

        # Adjust for current multiplier
        while (int(curr_hun) == 0):
            i %= limit
            limit /= 1000
            curr_hun = i // limit
            t += 1

        # Add hundreds' place if applicable
        if (int(curr_hun) > 99):
                # Explicitly converting the index to an integer
            hundreds_index = int(curr_hun) // 100
            answer += (first_twenty[hundreds_index] + " Hundred ")
        # Adjust for tens and ones
        curr_hun = int(curr_hun) % 100
        if (int(curr_hun) > 0 and int(curr_hun) < 20):
            answer += (first_twenty[int(curr_hun)] + " ")
        elif (int(curr_hun) % 10 == 0 and int(curr_hun) != 0):
                index = int(int(curr_hun) // 10) - 1
                answer += (tens[index] + " ")
        elif (int(curr_hun) > 19 and int(curr_hun) < 100):
                tens_index = int(int(curr_hun) // 10) - 1
                ones_index = int(int(curr_hun) % 10)
                answer += (tens[tens_index] + " " + first_twenty[ones_index] + " ")

        # Add the current multiplier
        if (t < 4):
            answer += (multiplier[t] + " ")

        i = i % limit
        limit = limit // 1000

    return answer



In [29]:
import re
import re
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')

def preprocess_text(text):
    

   # Convert numbers to words
    def replace_with_words(match):
        num = int(match.group())
        return numberToWords(num)

    text = re.sub(r'\b\d+\b', replace_with_words, text)

        # Replace punctuation
    text = re.sub(r'[^\w\s]', ' ', text)
    text = text.replace('+', ' plus ')

    # Apply lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in text.split()]
    text = ' '.join(lemmatized_words)

    return text


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\JellevanAs\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [30]:
# Apply preprocessing to each description
df_model['processed_descrip'] = df_model['descrip_en'].apply(preprocess_text)

In [31]:
from tqdm import tqdm
import numpy as np

embeddings = []

for text in tqdm(df_model['processed_descrip'], desc="Processing Texts"):
    words = text.split()  # Split the text into words
    word_embeddings = [word2vec_model[word] for word in words if word in word2vec_model.key_to_index]
    
    if word_embeddings:
        # Average the embeddings of all words in the text
        mean_embedding = np.mean(word_embeddings, axis=0)
    else:
        # If no words in the text are in the model's vocabulary, use a zero vector
        mean_embedding = np.zeros(word2vec_model.vector_size)

    embeddings.append(mean_embedding.tolist())

# Create a new column in the DataFrame for embeddings
df_model['word2vec_embeddings'] = embeddings


Processing Texts: 100%|██████████| 72014/72014 [01:11<00:00, 1005.75it/s]


In [32]:
from tqdm import tqdm


total_words = 0
embedded_words = 0

for text in tqdm(df_model['processed_descrip'], desc="Processing Texts"):
    words = text.split()  # Tokenize the text into words
    total_words += len(words)  # Count total words
    embedded_words += sum(word in word2vec_model.key_to_index for word in words)  # Count words in the model's vocabulary

print(f"Total words: {total_words}")
print(f"Embedded words: {embedded_words}")


Processing Texts: 100%|██████████| 72014/72014 [00:08<00:00, 8052.54it/s]

Total words: 37403038
Embedded words: 32645623


In [33]:
percentage_embedded = (embedded_words / total_words) * 100 if total_words > 0 else 0
print(percentage_embedded)

87.28067222774791


In [34]:
df_model.to_csv("word_2_vec.csv")